In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.append("/home/ubuntu/fastai/courses/dl1/fastai")

In [4]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [5]:
PATH='/home/ubuntu/course-shortcut/competitions/lyrics_generator/'
TRN = "/home/ubuntu/course-shortcut/competitions/lyrics_generator/all/trn/"
VAL = "/home/ubuntu/course-shortcut/competitions/lyrics_generator/all/val/"


In [17]:
lyrics = pd.read_csv("lyrics.csv")

In [8]:
lyrics.sample(n=10)

,index,song,year,artist,genre,lyrics
151535,151535,perfect,2005,alice-cooper,Rock,She's an overnight sensation\nIn the mirror on...
121140,121140,fleischwurstlied,1989,astrid-lindgren,Other,NaN
82400,82400,fanfare-darren-emerson-reprise,2016,c-jay,Electronic,NaN
227649,227649,i-feel-the-same,2002,bonnie-raitt,Rock,"Say goodbye,\nWell I know it's true.\nI know y..."
172086,172086,ladyhawke-theme,1985,andrew-powell,Rock,NaN
129288,129288,show-me-the-way,2006,destiny-s-child,Pop,"I wanna know, if i could get to know you\nSo m..."
56954,56954,algorithm,2016,emmy-the-great,Indie,NaN
221343,221343,slow-motion,2010,cold-flamez,Hip-Hop,Hold up\nSlow down little lady\nI want to feel...
147686,147686,christenings,2007,blackfield,Rock,I met you in a record store\nYou had slept in ...
122289,122289,love-for-sale,2014,donald-byrd,Jazz,NaN


In [9]:
lyrics['genre'].drop_duplicates()

0                 Pop
249           Hip-Hop
350     Not Available
363             Other
403              Rock
415             Metal
614           Country
659              Jazz
831        Electronic
860              Folk
1539              R&B
1643            Indie
Name: genre, dtype: object

In [10]:
len(lyrics[lyrics.genre == 'Hip-Hop'])

33965

In [14]:
def get_year_range(lyrics, start_year, end_year):
    filtered_lyrics = lyrics[start_year < lyrics.year]
    return filtered_lyrics[filtered_lyrics.year <= end_year]


In [15]:
hip_hop_lyrics = lyrics[lyrics.genre == 'Hip-Hop']
hip_hop_lyrics = get_year_range(hip_hop_lyrics, 2007, 2017)

NameError: name 'lyrics' is not defined

In [94]:
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [157]:
a, b = next(iter(hip_hop_lyrics.iterrows()))
b.lyrics

'Hey yo man, remember that shit Eazy did a while back\nMotherfuckers said it wasn\'t gonna work\nThat crazy shit, yeah the stupid shit\nHey yo Eazy! (Yo!)\nHey man why don\'t you come off the piano for a minute\nAnd bust this crazy shit\nWoke up quick at about noon\nJust thought that I had to be in Compton soon\nI gotta get drunk before the day begins\nBefore my mother starts bitchin\' about my friends\nAbout to go and damn near went blind\nYoung niggaz at the pad throwin\' up gang signs\nRan in the house and grabbed my clip\nWith the Mac-10 on the side of my hip\nBailed outside and pointed my weapon\nJust as I thought, the fools kept steppin\'\nJumped in the fo\' hit the juice on my ride\nI got front back and side to side\nThen I let the Alpine play\nBumpin\' new shit by NWA\nIt was "Gangsta Gangsta" at the top of the list\nThen I played my own shit, it went somethin\' like this\nCruisin\' down the street in my six-fo\'\nJockin\' the bitches, slappin\' the hoes\nWent to the park to ge

In [158]:
for (i,(_,r)) in enumerate(hip_hop_lyrics.iterrows()):
    dset = 'trn' if random.random()>0.1 else 'val'
    lyrics = r.lyrics
    from IPython.core.debugger import Tracer; Tracer()()

    if isinstance(lyrics, str) and not('mein' in lyrics):
        open(f'{PATH}all/{dset}/{i}.txt', 'w').write(str(lyrics))

/home/ubuntu/anaconda2/envs/environment.yml/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  after removing the cwd from sys.path.


> <ipython-input-158-67e166955ffa>(6)<module>()
      3     lyrics = r.lyrics
      4     from IPython.core.debugger import Tracer; Tracer()()
      5 
----> 6     if isinstance(lyrics, str) and not('mein' in lyrics):
      7         open(f'{PATH}all/{dset}/{i}.txt', 'w').write(str(lyrics))

ipdb> lyrics
'Hey yo man, remember that shit Eazy did a while back\nMotherfuckers said it wasn\'t gonna work\nThat crazy shit, yeah the stupid shit\nHey yo Eazy! (Yo!)\nHey man why don\'t you come off the piano for a minute\nAnd bust this crazy shit\nWoke up quick at about noon\nJust thought that I had to be in Compton soon\nI gotta get drunk before the day begins\nBefore my mother starts bitchin\' about my friends\nAbout to go and damn near went blind\nYoung niggaz at the pad throwin\' up gang signs\nRan in the house and grabbed my clip\nWith the Mac-10 on the side of my hip\nBailed outside and pointed my weapon\nJust as I thought, the fools kept steppin\'\nJumped in the fo\' hit the juice on my r

In [20]:
trn_files = !ls {TRN}
trn_files[:10]

['0.txt',
 '10000.txt',
 '10001.txt',
 '10002.txt',
 '10005.txt',
 '10007.txt',
 '10008.txt',
 '10009.txt',
 '10011.txt',
 '10012.txt']

In [39]:
from spacy.symbols import ORTH

my_tok = spacy.load('en')
def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

my_tok.tokenizer.add_special_case('\n', [{ORTH: '<NEWLINE>'}])

In [5]:
bs=64
bptt=300 #backpropogate through time: number of words it'll remember

In [8]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train=TRN, validation=VAL, test=VAL)
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

NameError: name 'my_spacy_tok' is not defined

In [6]:
bs=64
bptt=200 #backpropogate through time: number of words it'll remember
from spacy.symbols import ORTH
my_tok = spacy.load('en')
TEXT = pickle.load(open('/home/ubuntu/course-shortcut/competitions/lyrics_generator/models/TEXT.pkl', 'rb'))
FILES = dict(train=TRN, validation=VAL, test=VAL)
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [21]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(315, 16745, 1, 6074427)

### TRAINING 

In [8]:
em_sz = 200nh = 500 nl = 3optimization_function = partial(optim.Adam, betas=(0.7, 0.99))

In [9]:
learner = md.get_model(optimization_function, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)


In [14]:
learner.clip=0.5
learner.fit(3e-3, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       5.23085  5.1033 ]                                



In [15]:
sample_model(learner.model, "what", l=200)

NameError: name 'sample_model' is not defined

In [18]:
learner.clip=0.5
learner.fit(3e-3, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       4.85287  4.77611]                                



In [19]:
sample_model(learner.model, "what", l=200)

...you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what you want , what 

In [20]:
learner.clip=0.3
learner.fit(3e-3, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       4.66829  4.64223]                                



In [31]:
sample_model(learner.model, "I know the mark's high, butterflies rip apart my stomach", l=200)

...is waiting see you lost man , do n't act scared i said , but that 's right here , cause you 're built with played real , baby once 's a second wheeler , - cu - eh punk hoe . all i want was a beating back donald kelly on the departed palms all in my ear with a bedroom in the air , fiends cuff me me and me with leg land , from a have to call yo , i used to see myself flip the pool up till i 'm clearing trick , double bear caine , my niggas givin me for sack a challenge [ chorus ] you have kill me , leave me , remember spaced off totem , michael houses and was n't somehow pay hold on you teach <unk> for a hood again , high joints shooting toes after the sun , shout chill to my breath hey , stil used to confusin lyrics from the advance till i drop your streets forget you high man coz pressure sounds like a little ripper stuff ven tuesday , sings od tec eh pen , gras and <unk> cigarettes they do n't make 

In [32]:
learner.save("em_saved_12_50")

In [29]:
# learner.load('em_200_11_24')

def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])
model=learner.model

def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    

    for i in range(l):
        pred = res
        pred_last_prob = pred[-1].exp() / pred[-1].exp().sum()    
        # Randomly sample one word
        n = torch.multinomial(pred_last_prob, 1)
#         print(TEXT.vocab.itos[pred_last_choice.data[0]])
#         n=res[-1].topk(2)[1]
#         n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [12]:
learner.fit(3e-4, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       4.73076  4.73163]                                



In [13]:
sample_model(learner.model, "I know the mark's high, butterflies rip apart my stomach", l=200)

...is on my mind , i 'm a make you know i 'm a make you know i 'm a make you feel like i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm a get it , i 'm 

In [9]:
em_sz = 500nh = 500nl = 3
optimization_function = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(optimization_function, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

In [10]:
learner.clip=0.3
learner.fit(3e-3, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       4.92101  4.8376 ]                                



In [18]:
sample_model(learner.model, "when how", l=200)

...can you get it ? i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back , i 'm a get it back 

In [19]:
learner.save("em_size_500_bs_32")

In [20]:
learner.clip=0.3
learner.fit(3e-3, 1, wds=1e-7)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       4.60375  4.584  ]                                



In [23]:
sample_model(learner.model, "I like", l=200)

...to get it right , i 'm a get it right , i 'm a get it right i 'm a get it right , i 'm a get it right i 'm a get it right , get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right , get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now i 'm a get it right now 

In [11]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])
model=learner.model
s="hello"

In [12]:
learner.fit(3e-3, 1, wds=1e-6)

Widget Javascript not detected.  It may not be installed or enabled properly.


[ 0.       3.95085  4.23278]                                



In [12]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [13]:
sample_model(model, "test", l=200)

...i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , i 'm a , 

### TESTING

In [119]:
model=learner.model
ss="I'm beginning to feel like a Rap God, Rap God "
s = [spacy_tok(ss)]
input_text=TEXT.numericalize(s)
' '.join(s[0])

"I 'm beginning to feel like a Rap God , Rap God"

In [130]:
# Set batch size to 1
model[0].bs=1
# Turn off dropout
model.eval()
# Reset hidden state
model.reset()
# Get predictions from model
results,*_ = model(input_text)
# Put the batch size back to what it was
model[0].bs=bs

In [133]:
nexts = torch.topk(results[-1], 1)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['<unk>']

In [134]:
to_np(nexts)

array([0])

In [140]:
results[-1].topk

<bound method Variable.topk of Variable containing:
 7.6864
-2.8855
 7.2407
   ⋮   
-2.5771
-2.5315
-2.5787
[torch.cuda.FloatTensor of size 16745 (GPU 0)]
>

In [137]:
print(ss,"\n")
# nexts = torch.topk(res[-1], 30)[1]
# [TEXT.vocab.itos[o] for o in to_np(nexts)]
previous_inputs = ss
for i in range(100):
    n=results[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = model(n[0].unsqueeze(0))
print('...')

I'm beginning to feel like a Rap God, Rap God  

, i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast , i 'm a beast ...
